In [1]:
# Import libraries

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Acquire the data

In [3]:
# Load the curriculum access

df_log = pd.read_csv('anonymized-curriculum-access.txt',          
                      engine='python',
                      header=None,
                      index_col=False,
                      sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                      na_values='"-"',
)

df_log.head()

,0,1,2,3,4,5
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [4]:
df_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719459 entries, 0 to 719458
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       719459 non-null  object 
 1   1       719459 non-null  object 
 2   2       719458 non-null  object 
 3   3       719459 non-null  int64  
 4   4       674619 non-null  float64
 5   5       719459 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 32.9+ MB


In [5]:
# Load the cohort info

df_cohort = pd.read_csv('cohorts.csv')
df_cohort.head()

,cohort_id,name,start_date,end_date,program_id
0,1,Arches,2014-02-04,2014-04-22,1
1,2,Badlands,2014-06-04,2014-08-22,1
2,3,Carlsbad,2014-09-04,2014-11-05,1
3,4,Denali,2014-10-20,2015-01-18,1
4,5,Everglades,2014-11-18,2015-02-24,1


In [6]:
df_cohort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cohort_id   46 non-null     int64 
 1   name        46 non-null     object
 2   start_date  46 non-null     object
 3   end_date    46 non-null     object
 4   program_id  46 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 1.9+ KB


### Preapre the datasets for merge

In [7]:
# Rename the columns in df_log

df_log.columns = ['date', 'time', 'page_accessed', 'user_id', 'cohort_id', 'ip']
df_log.head()

,date,time,page_accessed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [8]:
# How many unique cohort_id in df_log?
df_log.cohort_id.nunique()  # only 40 unique cohort ids

40

In [9]:
# How many unique cohorts in df_cohort? should be 40
df_cohort.cohort_id.nunique() # 46 unique cohort_ids

46

In [10]:
# Which 6 cohort_ids are missing in df_log?

log_cohort_ids = df_log.cohort_id.value_counts().index
cohort_ids = df_cohort.cohort_id.value_counts().index
set(cohort_ids) - set(log_cohort_ids)

{3, 10, 20, 30, 60, 132}

In [11]:
# How many unique program_id in df_cohort?
df_cohort.program_id.value_counts()

2    26
1    14
3     4
4     2
Name: program_id, dtype: int64

**Takeaways**:
- df_cohort is pretty clean and don't need a lot of cleanining. 
- do I need to convert the start and end date to timestamp?

### Merge two datasets

In [12]:
# Print out the shapes of the dataframes
df_log.shape, df_cohort.shape

((719459, 6), (46, 5))

In [13]:
# Left join the two datasets on cohort ids
df = df_log.merge(df_cohort, how='left', on='cohort_id')

# Print the shape
df.shape

(719459, 10)

In [14]:
# Take a peek at the new dataframe
df.sample(5) # Matched

,date,time,page_accessed,user_id,cohort_id,ip,name,start_date,end_date,program_id
598508,2020-07-14,15:32:10,mysql/tables,595,56.0,216.63.31.195,Apex,2020-02-24,2020-07-29,2.0
211527,2019-03-03,23:10:08,html-css/introduction,341,29.0,204.44.112.76,Zion,2019-01-22,2019-06-04,2.0
167817,2018-12-10,10:49:31,jquery/essential-methods/attributes-and-css,312,28.0,97.105.19.58,Staff,2014-02-04,2014-02-04,2.0
426525,2020-01-09,13:32:33,examples/javascript/blog.json,535,52.0,97.105.19.58,Europa,2019-11-04,2020-04-17,2.0
345763,2019-09-10,13:36:51,3-sql/2-mysql-introduction,481,34.0,97.105.19.58,Bayes,2019-08-19,2020-01-30,3.0


In [15]:
# Quick check: there should be 40 uniuqe cohort ids in the new dataframe
df.cohort_id.nunique() # matched

40

### Handle date and time columns
- Combines access date and time and set it as the index
- Convert the start and end dates to datetime dtypes

In [16]:
# Create a new column for timestamp
df['timestamp'] = df.date.str.cat(df.time, sep=' ')

# Conver to datetime dtype
df.timestamp = pd.to_datetime(df.timestamp)

# Set the timestamp as index
df = df.set_index('timestamp').sort_index()

# Drop columns: date and time
df.drop(columns=['date', 'time'], inplace=True)

# Take peek at the df
df.head()

,page_accessed,user_id,cohort_id,ip,name,start_date,end_date,program_id
timestamp,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0


In [17]:
# Print the dtypes of the colunms
df.dtypes

page_accessed     object
user_id            int64
cohort_id        float64
ip                object
name              object
start_date        object
end_date          object
program_id       float64
dtype: object

In [18]:
# Convert the start and end dates to datetime dtpye

df.start_date = pd.to_datetime(df.start_date)
df.end_date = pd.to_datetime(df.end_date)

# Print the dtypes
df.dtypes

page_accessed            object
user_id                   int64
cohort_id               float64
ip                       object
name                     object
start_date       datetime64[ns]
end_date         datetime64[ns]
program_id              float64
dtype: object

### Break down the page information to lessons

In [58]:
# Create a variable to hold page_accessed
df_pages = df.page_accessed
df_pages.head()

timestamp
2018-01-26 09:55:03                                      /
2018-01-26 09:56:02                                java-ii
2018-01-26 09:56:05    java-ii/object-oriented-programming
2018-01-26 09:56:06     slides/object_oriented_programming
2018-01-26 09:56:24              javascript-i/conditionals
Name: page_accessed, dtype: object

In [59]:
# Replace / with homepage
df_pages.replace('/', 'homepage', inplace=True)

In [60]:
# Split the url by the first '/' and expand to two columns
df_pages = df_pages.str.split('/', n=1, expand=True)

# Take a peek at the new df
df_pages.head()

,0,1
timestamp,,
2018-01-26 09:55:03,homepage,None
2018-01-26 09:56:02,java-ii,None
2018-01-26 09:56:05,java-ii,object-oriented-programming
2018-01-26 09:56:06,slides,object_oriented_programming
2018-01-26 09:56:24,javascript-i,conditionals


In [61]:
# Change the columns names
df_pages.columns = ['lesson', 'sublesson']

In [63]:
# Concat the lesson columns to the original dataframe
df = pd.concat([df, df_pages], axis=1)

# Drop the column page_accessed
df.drop(columns='page_accessed', inplace=True)

# Print the shape
df.shape

(719459, 9)

In [64]:
# Take a peek at the new dataframe
df.head()

,user_id,cohort_id,ip,name,start_date,end_date,program_id,lesson,sublesson
timestamp,,,,,,,,,
2018-01-26 09:55:03,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,homepage,None
2018-01-26 09:56:02,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,java-ii,None
2018-01-26 09:56:05,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,java-ii,object-oriented-programming
2018-01-26 09:56:06,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,slides,object_oriented_programming
2018-01-26 09:56:24,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0,javascript-i,conditionals


In [65]:
# Quick summary
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 719459 entries, 2018-01-26 09:55:03 to 2020-11-02 16:48:47
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     719459 non-null  int64         
 1   cohort_id   674619 non-null  float64       
 2   ip          719459 non-null  object        
 3   name        674619 non-null  object        
 4   start_date  674619 non-null  datetime64[ns]
 5   end_date    674619 non-null  datetime64[ns]
 6   program_id  674619 non-null  float64       
 7   lesson      719458 non-null  object        
 8   sublesson   541536 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 54.9+ MB


**Takeaways**
- Null values are found in the columns below:
    - cohort_id
    - cohort_name
    - start_date
    - end_date
    - program_id
    - lesson
    - sublesson

### Build helper function